# Heavy Tailed Universality Classes

We demonstrate how the Heavy Tailed / Power Law Universality Classes work

for matrix elements  $M_{i,j}=x$ we choose  $p(x)\sim\dfrac{1}{x^{1+\mu}}$

and we fit the ESD $p(\lambda)\sim\dfrac{1}{\lambda^{1+\mu}}$

We ask, what is $\mu$ (for the Matrix) given a fit for $\alpha$ (from the ESD)


### Truncated Power Law $\mu>2$

 $$p(\lambda)\sim\dfrac{1}{\lambda^{1+\mu}}$$
 
 $$\alpha=1+\mu$$


example: $\mu=2.5;\;\;\alpha=1+\mu=3.5$


### Power Law $\mu<2$

 $$p(\lambda)\sim\dfrac{1}{\lambda^{1+\mu/2}}$$
 
  $$\alpha=2(1+\mu)$$



example: $\mu=1;\;\;\alpha=1+\dfrac{\mu}{2}=1.5$

### PowerLaw fitting code

https://github.com/jeffalstott/powerlaw

### References

and see page 26 of 

http://www-syscom.univ-mlv.fr/~najim/gdr/bouchaud.pdf

and the actual proof

https://arxiv.org/pdf/0811.1587.pdf


In [1]:
import numpy as np
import tensorflow as tf
import pickle, time
from copy import deepcopy
from shutil import copy
from tqdm import tqdm_notebook as tqdm
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import import_ipynb
import RMT_Util
import powerlaw

import sklearn
from sklearn.decomposition import TruncatedSVD
from sklearn.random_projection import sparse_random_matrix
print(sklearn.__version__)

/Users/charlesmartin14/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/Users/charlesmartin14/.local/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


importing notebook from RMT_Util.ipynb


Using TensorFlow backend.


3.6.5 |Anaconda custom (64-bit)| (default, Apr 26 2018, 08:42:37) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
numpy version 1.14.2
scipy version 1.0.1
tensforflow version 1.5.0-dev20171116
keras version 2.1.5
sklearn version 0.19.1
powerlaw version 1.4.3
0.19.1


### create heavy tailed Pareto distibution with different $\mu$, and fit with PowerLaw


Note: for python Pareto function, $a=\mu$ 

#### First, test the estimator on the actual distribution


In [ ]:
alphas = {}
for mu in [0.5, 1.0, 1.5, 1.9, 2.1, 2.5, 3.0, 4.0, 5.0]:
      
    data = np.random.pareto(mu,size=10000)
    
    fit = powerlaw.Fit(data)#, xmax=np.max(data))
    fig2 = fit.plot_pdf(color='b', linewidth=2)
    fit.power_law.plot_pdf(color='b', linestyle='--', ax=fig2)
    plt.show()
    
    bins = np.logspace(np.log10(np.min(data)), np.log10(np.max(data)), num=100)
    plt.title("mu+1 = {:3g} alpha = {:3g} D = {:3g}".format(mu+1, fit.alpha, fit.D))
    plt.hist(data, bins=bins, density=True, log=True, color='blue', alpha=0.5);
    plt.xscale('log')
    plt.show()
    
    alphas[mu]=fit.alpha
    

In [ ]:
x, y = [], []
for mu, alpha in alphas.items():
    x.append(mu)
    y.append(alpha)
    
plt.scatter(x,y)   
plt.xlabel('$\mu$')
plt.ylabel(r'$\alpha=1+\mu$')
plt.title(r'Powerlaw estimate for $\alpha$ given $\mu$ ')

#### Now test on matrix $\mathbf{W}$ and its ESD , from $\mathbf{X}=\mathbf{W}^{T}\mathbf{W}$

In [4]:
def mu4alpha(alpha):
    if (alpha >= 2.0 and alpha <= 3.0):
        mu = 2.0
    elif (alpha < 2.0):
        mu = 2.0*(alpha - 1.0)
    elif (alpha > 3.0):
        mu = alpha - 1.0
    return mu

In [2]:
def get_pareto_mat(mu, M=10000, Q=1):
    W = np.random.pareto(mu,size=(M,int(Q*M)))
    Q = np.max(W.shape)/np.min(W.shape)
    #evals, _ = RMT_Util.eigenspectrum(W)
    
    X = np.dot(W,W.T)
    svd = TruncatedSVD(n_components=M-1, n_iter=7, random_state=42)
    svd.fit(X)  
    sv = svd.singular_values_
    evals = sv*sv

    return W, evals

In [3]:
alphas4W = {}
alphas4Evals = {}

for mu in [0.5, 1.0, 1.5, 1.9, 2.1, 2.5, 3.0, 4.0, 5.0]:
      
    W, evals = get_pareto_mat(mu)
    data = np.ndarray.flatten(W)
    fit = powerlaw.Fit(data, xmax=np.max(data))
    
    alphas4W[mu]=fit.alpha

    data = evals
    fit = powerlaw.Fit(data, xmax=np.max(data))
    fig2 = fit.plot_pdf(color='b', linewidth=2)
    fit.power_law.plot_pdf(color='b', linestyle='--', ax=fig2)
    plt.show()
    
    mu4alpha = mu4alpha(fit.alpha)
    alphas4Evals[mu]=fit.alpha

    bins = np.logspace(np.log10(np.min(data)), np.log10(np.max(data)), num=100)
    plt.title("mu = {:3g} alpha = {:3g} mu4alpha {:3g} D = {:3g} ".format(mu, fit.alpha, mu4alpha, fit.D))
    plt.hist(data, bins=bins, density=True, log=True, color='blue', alpha=0.5);
    plt.xscale('log')
    plt.show()
    


Calculating best minimal value for power law fit
/Users/charlesmartin14/anaconda3/lib/python3.6/site-packages/powerlaw.py:697: RuntimeWarning: divide by zero encountered in true_divide
  (Theoretical_CDF * (1 - Theoretical_CDF))
/Users/charlesmartin14/anaconda3/lib/python3.6/site-packages/powerlaw.py:697: RuntimeWarning: invalid value encountered in true_divide
  (Theoretical_CDF * (1 - Theoretical_CDF))


KeyboardInterrupt: 

In [ ]:
x, y = [], []
for mu, alpha in alphas4W.items():
    x.append(mu)
    y.append(alpha)
    
plt.scatter(x,y)   
plt.xlabel('$\mu$')
plt.ylabel(r'$\alpha=1+\mu$')
plt.title(r'Powerlaw estimate for $\alpha$ given Matrix $\mathbf{W}(\mu)$')

In [ ]:
x, y = [], []
for mu, alpha in alphas4Evals.items():
    x.append(mu)
    y.append(alpha)
    
plt.scatter(x,y)   
plt.xlabel('$\mu$')
plt.ylabel(r'$\alpha=1+\mu$')
plt.title(r'Powerlaw estimate for $\alpha$ ESD from $\mathbf{W}(\mu)$')